In [ ]:
%pip install pandas,numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: 'pandas,numpy': Expected end or semicolon (after name and no valid version specifier)
    pandas,numpy
          ^


In [ ]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#CSV Import
import pandas as pd
import numpy as np
#df_Projects=pd.read_excel('C:/Users/xuanchunhuang/OneDrive - KPMG/Documents/Digi/project_user.xlsx',sheet_name='Projects')
#df_Users=pd.read_excel('C:/Users/xuanchunhuang/OneDrive - KPMG/Documents/Digi/project_user.xlsx',sheet_name='Users')
df_Projects=pd.read_excel('project_user.xlsx',sheet_name='Projects')
df_Users=pd.read_excel('project_user.xlsx',sheet_name='Users')
df_Projects.head()

,Project Name,Project Creator,Opportunity Name,Opportunity Type,No. of EoI Submissions,Submission Due Date,Status,Collaborated Partner
0,Energy Transition for Corporate Clients,Energi Tranz Pte Ltd,ESG Reporting Support for Businesses,Experts/Professional Services,0,"Jan 31, 2026",Open,NaN
1,Energy Transition for Corporate Clients,Energi Tranz Pte Ltd,Energy Transition Strategy for Corporate Clients,Experts/Professional Services,1,"Apr 30, 2026",Closed,Reetz
2,Optimizing Facility Ops with BESS,Energetix Pte Ltd,Peak Demand Management & Cost Savings,Innovation/Ideation,1,"Apr 30, 2026",Closed,Blue Whale Energy Pte. Ltd.
3,Optimizing Facility Ops with BESS,Energetix Pte Ltd,Backup Power During Outages,Solutions,1,"May 31, 2026",Closed,Pixii
4,Optimizing Facility Ops with BESS,Energetix Pte Ltd,Renewable Energy Integration,Solutions,0,"Apr 29, 2026",Open,NaN


In [4]:
# exclude some keywords(Collaborated Partner)
exclude_keywords = ['Branch', 'department', 'division']

def should_keep(partner):
    lower = partner.lower()
    return not any(kw in lower for kw in exclude_keywords)

# spliting and filtering
def split_partners(row):
    raw = row['Collaborated Partner']
    if isinstance(raw, str):
        parts = [p.strip() for p in raw.split(',') if p.strip()]
        return [p for p in parts if should_keep(p)]
    return []

#project and engaged user
project_long = []
for idx, row in df_Projects.iterrows():
    partners = split_partners(row)
    for p in partners:
        project_long.append({'Project Name': row['Project Name'], 'partner': p})
project_long_df = pd.DataFrame(project_long)
project_long_df

,Project Name,partner
0,Energy Transition for Corporate Clients,Reetz
1,Optimizing Facility Ops with BESS,Blue Whale Energy Pte. Ltd.
2,Optimizing Facility Ops with BESS,Pixii
3,BESS Opportunities,Blue Whale Energy Pte. Ltd.
4,Synergy opportunities (PV),GetSolar
5,AI Solutions for Industrial Sectors,Pantherpulse Pte Ltd
6,Carbon Management Solutions (Aviation),Unravel Carbon
7,Facilitate Integration in Energy market,Energetix Pte Ltd
8,Solar PV Optimisation,PV Doctor Pte Ltd
9,Solar PV Optimisation,Blue Whale Energy Pte. Ltd.


In [5]:
# exclude some keywords(User Name)
exclude_keywords = ['Branch', 'department', 'division']

def should_keep_name(name):
    if not isinstance(name, str):
        return False
    lower = name.lower()
    return not any(kw in lower for kw in exclude_keywords)

df_Users = df_Users[df_Users['User Name'].apply(should_keep_name)].copy()

#Standardize
import re
def standardize(text):
    if not isinstance(text, str):
        return ''
    text = text.strip()
    text = re.sub(r'Pte\s*\.?\s*Ltd\.?', 'Pte. Ltd.', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text)
    return text.lower()
df_Users['std_name'] = df_Users['User Name'].apply(standardize)
project_long_df['std_partner'] = project_long_df['partner'].apply(standardize)

# whether engage
engaged = set(zip(project_long_df['Project Name'], project_long_df['std_partner']))

rows = []
for _, user in df_Users.iterrows():
    for _, project in df_Projects.iterrows():
        key = (project['Project Name'], user['std_name'])
        is_engaged = int(key in engaged)
        rows.append({
            'user_id': user['User Name'],
            'project_id': project['Project Name'],
            'is_engaged': is_engaged,
            'User Type': user['User Type'],
            'Location': user['Location'],
            'Sector': user['Sector'],
            'Project Creator': project['Project Creator'],
            'Opportunity Type': project['Opportunity Type'],
            'No. of EoI Submissions': project['No. of EoI Submissions'],
            'Status': project['Status']
        })

engagement_df = pd.DataFrame(rows)
engagement_df

,user_id,project_id,is_engaged,User Type,Location,Sector,Project Creator,Opportunity Type,No. of EoI Submissions,Status
0,EDF HQ Singapore,Energy Transition for Corporate Clients,0,COMPANY,Singapore,Energy,Energi Tranz Pte Ltd,Experts/Professional Services,0,Open
1,EDF HQ Singapore,Energy Transition for Corporate Clients,0,COMPANY,Singapore,Energy,Energi Tranz Pte Ltd,Experts/Professional Services,1,Closed
2,EDF HQ Singapore,Optimizing Facility Ops with BESS,0,COMPANY,Singapore,Energy,Energetix Pte Ltd,Innovation/Ideation,1,Closed
3,EDF HQ Singapore,Optimizing Facility Ops with BESS,0,COMPANY,Singapore,Energy,Energetix Pte Ltd,Solutions,1,Closed
4,EDF HQ Singapore,Optimizing Facility Ops with BESS,0,COMPANY,Singapore,Energy,Energetix Pte Ltd,Solutions,0,Open
...,...,...,...,...,...,...,...,...,...,...
25896,Kai Ling,Rooftop Solar Deployment PPA,0,INDIVIDUAL,NaN,NaN,Energi Tranz Pte Ltd,Solutions,1,Closed
25897,Kai Ling,CorpCarbon Emission Reporting App,0,INDIVIDUAL,NaN,NaN,Energi Tranz Pte Ltd,Experts/Professional Services,1,Closed
25898,Kai Ling,Aligned Energy Bintan Green Diesel Manufacturing,0,INDIVIDUAL,NaN,NaN,Aligned Energy Pte Ltd,Financing,0,Closed
25899,Kai Ling,Prototyping of BIPV modules,0,INDIVIDUAL,NaN,NaN,Solar Energy Research Institute of SG (SERIS),Innovation/Ideation,1,Closed


Logistic Regression

In [6]:
training_df = engagement_df[engagement_df['Status'] == 'Closed'].copy()

In [7]:
X = training_df.drop(columns=['user_id', 'project_id', 'is_engaged', 'Status'])
y = training_df['is_engaged']

In [8]:
%pip sklearn.pipeline
%pip sklearn.compose
%pip sklearn.preprocessing
%pip sklearn.linear_model

ERROR: unknown command "sklearn.pipeline"
ERROR: unknown command "sklearn.compose"
ERROR: unknown command "sklearn.preprocessing"
ERROR: unknown command "sklearn.linear_model"


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

cat_cols = ['User Type', 'Location', 'Sector', 'Project Creator', 'Opportunity Type']
num_cols = ['No. of EoI Submissions']

preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], remainder='passthrough')

model = Pipeline([
    ('prep', preprocessor),
    ('logreg', LogisticRegression())
])

In [10]:
model.fit(X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('prep',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['User Type', 'Location',
                                                   'Sector', 'Project Creator',
                                                   'Opportunity Type'])])),
                ('logreg', LogisticRegression())])

In [11]:
# Create df for predicting
open_df = engagement_df[engagement_df['Status'] == 'Open'].copy()
feature_cols = ['User Type', 'Location', 'Sector', 'Project Creator', 'Opportunity Type', 'No. of EoI Submissions']
X_open = open_df[feature_cols]

In [94]:
# Calculate engagement probability for each customer towards all open projects
open_df['engage_prob'] = model.predict_proba(X_open)[:, 1]
open_df

,user_id,project_id,is_engaged,User Type,Location,Sector,Project Creator,Opportunity Type,No. of EoI Submissions,Status,engage_prob
0,EDF HQ Singapore,Energy Transition for Corporate Clients,0,COMPANY,Singapore,Energy,Energi Tranz Pte Ltd,Experts/Professional Services,0,Open,0.009677
4,EDF HQ Singapore,Optimizing Facility Ops with BESS,0,COMPANY,Singapore,Energy,Energetix Pte Ltd,Solutions,0,Open,0.017089
5,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,0,COMPANY,Singapore,Energy,Sustainable Energy Association of Singapore SEAS,Innovation/Ideation,0,Open,0.013324
6,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,0,COMPANY,Singapore,Energy,Sustainable Energy Association of Singapore SEAS,Innovation/Ideation,0,Open,0.013324
7,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,0,COMPANY,Singapore,Energy,Sustainable Energy Association of Singapore SEAS,Innovation/Ideation,0,Open,0.013324
...,...,...,...,...,...,...,...,...,...,...,...
25869,Kai Ling,Regional Solar and Wind Deployment Accelerator,0,INDIVIDUAL,NaN,NaN,Rystad Energy Asia Pte. Ltd.,Solutions,0,Open,0.000138
25871,Kai Ling,UPS Storage Reduction Project,0,INDIVIDUAL,NaN,NaN,Reetz,Solutions,0,Open,0.000187
25874,Kai Ling,Searching for Sustainable Solutions for Aviation,0,INDIVIDUAL,NaN,NaN,Reetz,Solutions,0,Open,0.000187
25877,Kai Ling,Financing for Solar PV System Development (SG),0,INDIVIDUAL,NaN,NaN,Northmore Gordon Pte Ltd,Solutions,0,Open,0.000141


In [13]:
# Example score for 'EDF HQ Singapore'
open_df[open_df['user_id'] == 'EDF HQ Singapore'].sort_values(by='engage_prob', ascending=False).head(5)


,user_id,project_id,is_engaged,User Type,Location,Sector,Project Creator,Opportunity Type,No. of EoI Submissions,Status,engage_prob
4,EDF HQ Singapore,Optimizing Facility Ops with BESS,0,COMPANY,Singapore,Energy,Energetix Pte Ltd,Solutions,0,Open,0.017089
14,EDF HQ Singapore,Carbon Management Solutions (Aviation),0,COMPANY,Singapore,Energy,Reetz,Innovation/Ideation,0,Open,0.015237
26,EDF HQ Singapore,Aviation UPS Project,0,COMPANY,Singapore,Energy,Reetz,Innovation/Ideation,0,Open,0.015237
6,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,0,COMPANY,Singapore,Energy,Sustainable Energy Association of Singapore SEAS,Innovation/Ideation,0,Open,0.013324
7,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,0,COMPANY,Singapore,Energy,Sustainable Energy Association of Singapore SEAS,Innovation/Ideation,0,Open,0.013324


Association Analysis

In [27]:
# Get engagement records
engaged_df = engagement_df[engagement_df['is_engaged'] == 1].copy()
def submission_level(val):
    if val == 0:
        return 'submission=low'
    elif val <= 2:
        return 'submission=mid'
    else:
        return 'submission=high'

engaged_df['submission_level'] = engaged_df['No. of EoI Submissions'].apply(submission_level)

# 把每一行转为一组 item
def make_items(row):
    return [
        f"user_type={row['User Type']}",
        f"location={row['Location']}",
        f"sector={row['Sector']}",
        f"project_creator={row['Project Creator']}",
        f"opp_type={row['Opportunity Type']}",
        row['submission_level']
    ]

transactions = engaged_df.apply(make_items, axis=1).tolist()

In [32]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_tf = pd.DataFrame(te_array, columns=te.columns_)

# frequent itemsets
frequent_itemsets = apriori(df_tf, min_support=0.01, use_colnames=True)

# Rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(opp_type=Experts/Professional Services),(location=Singapore),0.260274,0.945205,0.246575,0.947368,1.002288,1.0,0.000563,1.041096,0.003086,0.257143,0.039474,0.604119
1,(opp_type=Financing),(location=Singapore),0.013699,0.945205,0.013699,1.000000,1.057971,1.0,0.000751,inf,0.055556,0.014493,1.000000,0.507246
2,(opp_type=Innovation/Ideation),(location=Singapore),0.123288,0.945205,0.109589,0.888889,0.940419,1.0,-0.006943,0.493151,-0.067395,0.114286,-1.027778,0.502415
3,(location=Singapore),(opp_type=Solutions),0.945205,0.602740,0.575342,0.608696,1.009881,1.0,0.005630,1.015221,0.178571,0.591549,0.014993,0.781621
4,(opp_type=Solutions),(location=Singapore),0.602740,0.945205,0.575342,0.954545,1.009881,1.0,0.005630,1.205479,0.024631,0.591549,0.170455,0.781621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10310,"(user_type=INDIVIDUAL, location=nan)","(submission=mid, sector=nan, project_creator=R...",0.027397,0.068493,0.027397,1.000000,14.600000,1.0,0.025521,inf,0.957746,0.400000,1.000000,0.700000
10311,"(user_type=INDIVIDUAL, sector=nan)","(submission=mid, location=nan, project_creator...",0.027397,0.027397,0.027397,1.000000,36.500000,1.0,0.026647,inf,1.000000,1.000000,1.000000,1.000000
10312,"(location=nan, sector=nan)","(user_type=INDIVIDUAL, submission=mid, project...",0.041096,0.027397,0.027397,0.666667,24.333333,1.0,0.026271,2.917808,1.000000,0.666667,0.657277,0.833333
10313,(user_type=INDIVIDUAL),"(project_creator=Reetz , submission=mid, opp_t...",0.041096,0.027397,0.027397,0.666667,24.333333,1.0,0.026271,2.917808,1.000000,0.666667,0.657277,0.833333


In [86]:
def recommend_for_user(user_row, project_df, rules):
    # Construct User attributes
    user_features = {
        f"user_type={user_row['User Type']}",
        f"location={user_row['Location']}",
        f"sector={user_row['Sector']}"
    }

    # select matched rules: antecedent (User attributes)
    matched_rules = rules[rules['antecedents'].apply(lambda x: x.issubset(user_features))]

    # Select“Open”Projects
    candidate_projects = project_df[project_df['Status'] == 'Open'].copy()

    def get_project_scores(row):
        # Project attributes
        project_attrs = {
            f"opp_type={row['Opportunity Type']}",
            f"project_creator={row['Project Creator']}",
            f"submission={submission_level(row['No. of EoI Submissions'])}"
        }

        # The hit-rules' confidence
        hit_confidences = []
        for idx, conseq in enumerate(matched_rules['consequents']):
            if not conseq.isdisjoint(project_attrs):
                hit_confidences.append(matched_rules.iloc[idx]['confidence'])

        rule_hits = len(hit_confidences)
        max_conf = max(hit_confidences) if hit_confidences else 0
        final_score = rule_hits + 0.5 * max_conf # Rule hits more important
        return pd.Series({'rule_hits': rule_hits, 'max_conf': max_conf,'final_score': final_score})

    # score index
    scores = candidate_projects.apply(get_project_scores, axis=1)
    candidate_projects = pd.concat([candidate_projects, scores], axis=1)
    candidate_projects['User Name'] = user_row['User Name']

    # more hit-rules, higher score, if same quantity, higher max_confidence, higher score
    candidate_projects = candidate_projects.sort_values(by='final_score', ascending=False)

    return candidate_projects

In [91]:
all_users = df_Users['User Name'].unique()
open_project_df = df_Projects[df_Projects['Status'] == 'Open'].copy()
final_recommendations = []
for user_id in all_users:
    user_row = df_Users[df_Users['User Name'] == user_id].iloc[0]
    rec_df = recommend_for_user(user_row, open_project_df, rules)  # 用包含项目和属性的 open_df
    rec_df['User Name'] = user_id  # 加上用户 ID
    final_recommendations.append(rec_df)
final_df = pd.concat(final_recommendations, ignore_index=True)

In [92]:
final_df

,Project Name,Project Creator,Opportunity Name,Opportunity Type,No. of EoI Submissions,Submission Due Date,Status,Collaborated Partner,rule_hits,max_conf,final_score,User Name
0,Optimizing Facility Ops with BESS,Energetix Pte Ltd,Renewable Energy Integration,Solutions,0,"Apr 29, 2026",Open,NaN,28.0,0.703704,28.351852,EDF HQ Singapore
1,Carbon Footprint Calculation & ESG Reporting,ESGpedia Pte Ltd,Carbon Calculation and ESG Reporting,Solutions,0,"Apr 30, 2026",Open,NaN,28.0,0.703704,28.351852,EDF HQ Singapore
2,UPS Storage Reduction Project,Reetz,Energy Efficiency Assessment:,Solutions,0,"Sep 30, 2025",Open,NaN,28.0,0.703704,28.351852,EDF HQ Singapore
3,Searching for Sustainable Solutions for Aviation,Reetz,Looking for collaborators for RECs,Solutions,0,"Sep 30, 2025",Open,NaN,28.0,0.703704,28.351852,EDF HQ Singapore
4,Regional Solar and Wind Deployment Accelerator,Rystad Energy Asia Pte. Ltd.,Driving Regional Energy Independence,Solutions,0,"Sep 30, 2025",Open,NaN,28.0,0.703704,28.351852,EDF HQ Singapore
...,...,...,...,...,...,...,...,...,...,...,...,...
6043,Solar Panels Recycling Project,Reetz,Financing Options,Financing,0,"Apr 30, 2025",Open,NaN,67.0,1.000000,67.500000,Kai Ling
6044,Energy Transition for Corporate Clients,Energi Tranz Pte Ltd,ESG Reporting Support for Businesses,Experts/Professional Services,0,"Jan 31, 2026",Open,NaN,24.0,0.333333,24.166667,Kai Ling
6045,Ministry of Sustainability & the Environment (...,Sustainable Energy Association of Singapore SEAS,Go Green SG,Innovation/Ideation,0,"Jun 30, 2025",Open,NaN,8.0,0.333333,8.166667,Kai Ling
6046,Ministry of Sustainability & the Environment (...,Sustainable Energy Association of Singapore SEAS,Singapore Green Plan,Innovation/Ideation,0,"Dec 31, 2029",Open,NaN,8.0,0.333333,8.166667,Kai Ling


In [90]:
user_row = df_Users[df_Users['User Name'] == 'EDF HQ Singapore'].iloc[0]
open_project_df = df_Projects[df_Projects['Status'] == 'Open'].copy()
recommended_projects = recommend_for_user(user_row, open_project_df, rules)
recommended_projects[['User Name','Project Name', 'Opportunity Type', 'Project Creator', 'No. of EoI Submissions', 'Status','rule_hits', 'max_conf','final_score']]

,User Name,Project Name,Opportunity Type,Project Creator,No. of EoI Submissions,Status,rule_hits,max_conf,final_score
4,EDF HQ Singapore,Optimizing Facility Ops with BESS,Solutions,Energetix Pte Ltd,0,Open,28.0,0.703704,28.351852
8,EDF HQ Singapore,Carbon Footprint Calculation & ESG Reporting,Solutions,ESGpedia Pte Ltd,0,Open,28.0,0.703704,28.351852
29,EDF HQ Singapore,UPS Storage Reduction Project,Solutions,Reetz,0,Open,28.0,0.703704,28.351852
32,EDF HQ Singapore,Searching for Sustainable Solutions for Aviation,Solutions,Reetz,0,Open,28.0,0.703704,28.351852
27,EDF HQ Singapore,Regional Solar and Wind Deployment Accelerator,Solutions,Rystad Energy Asia Pte. Ltd.,0,Open,28.0,0.703704,28.351852
11,EDF HQ Singapore,Synergy opportunities (PV),Solutions,Staubli Singapore Pte. Ltd.,0,Open,28.0,0.703704,28.351852
35,EDF HQ Singapore,Financing for Solar PV System Development (SG),Solutions,Northmore Gordon Pte Ltd,0,Open,28.0,0.703704,28.351852
6,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,Innovation/Ideation,Sustainable Energy Association of Singapore SEAS,0,Open,0.0,0.000000,0.000000
5,EDF HQ Singapore,Ministry of Sustainability & the Environment (...,Innovation/Ideation,Sustainable Energy Association of Singapore SEAS,0,Open,0.0,0.000000,0.000000
0,EDF HQ Singapore,Energy Transition for Corporate Clients,Experts/Professional Services,Energi Tranz Pte Ltd,0,Open,0.0,0.000000,0.000000


Final Weight

In [96]:
# Combine results of two models ['user_id', 'project_id']
final_df = final_df.rename(columns={'User Name': 'user_id','Project Name': 'project_id'})
combined_df = pd.merge(final_df, open_df, on=['user_id', 'project_id'], how='inner')

# Calculate weighted_score
combined_df['weighted_score'] = 0.4 * combined_df['final_score'] + 0.6 * combined_df['engage_prob']

In [97]:
top_n_per_user = (
    combined_df
    .sort_values(by=['user_id', 'weighted_score'], ascending=[True, False])
    .groupby('user_id')
    .head(5)
    .reset_index(drop=True)
)


In [99]:
top_n_per_user[['user_id', 'project_id', 'weighted_score', 'final_score', 'engage_prob']]

,user_id,project_id,weighted_score,final_score,engage_prob
0,Sivaraman,UPS Storage Reduction Project,42.200112,105.5,0.000187
1,Sivaraman,Searching for Sustainable Solutions for Aviation,42.200112,105.5,0.000187
2,Sivaraman,Aviation UPS Project,30.200131,75.5,0.000218
3,Sivaraman,Carbon Management Solutions (Aviation),30.200131,75.5,0.000218
4,Sivaraman,Optimizing Facility Ops with BESS,28.200147,70.5,0.000245
...,...,...,...,...,...
2155,xin,UPS Storage Reduction Project,42.200112,105.5,0.000187
2156,xin,Searching for Sustainable Solutions for Aviation,42.200112,105.5,0.000187
2157,xin,Aviation UPS Project,30.200131,75.5,0.000218
2158,xin,Carbon Management Solutions (Aviation),30.200131,75.5,0.000218
